## Modeling (Baseline Model: Seasonal Naive Forecast)
the forecast for any given day is the value from the same day in the previous week.

In [ ]:
from sklearn.metrics import mean_absolute_error

In [ ]:
# 1. Split data chronologically
train = df[df.index < "2025-03-01"]
test = df[df.index >= "2025-03-01"]

In [ ]:
# 2. Create the baseline forecast
test_dates = test.index
# We need the last 7 days of the training set to start the forecast
history = train["Incoming Calls"]

In [ ]:
# Create predictions with Seasonal Naive (Weekly) - handle missing dates
predictions = []
actual_test_dates = []

for date in test_dates:
    # Find the date from 7 days ago
    last_week_date = date - pd.Timedelta(days=7)

    # Check if the date exists in the training data
    if last_week_date in history.index:
        prediction = history.loc[last_week_date]
        predictions.append(prediction)
        actual_test_dates.append(date)

# Create a series with the predictions for evaluation
predictions_series = pd.Series(predictions, index=actual_test_dates)
actual_values = test.loc[actual_test_dates, "Incoming Calls"]

print(f"Number of predictions made: {len(predictions)}")
print("Predictions (first 10):")
print(predictions_series.head(10))

In [ ]:
# 3. Evaluate the baseline model
import numpy as np
from sklearn.metrics import mean_absolute_percentage_error, mean_squared_error

# Calculate evaluation metrics
mae = mean_absolute_error(actual_values, predictions_series)
rmse = np.sqrt(mean_squared_error(actual_values, predictions_series))
mape = mean_absolute_percentage_error(actual_values, predictions_series)

print("Seasonal Naive (Weekly) Baseline Model - Evaluation Metrics")
print("=" * 55)
print(f"Mean Absolute Error (MAE):           {mae:.2f}")
print(f"Root Mean Squared Error (RMSE):      {rmse:.2f}")
print(f"Mean Absolute Percentage Error (MAPE): {mape:.4f} ({mape*100:.2f}%)")
print("=" * 55)

In [ ]:
# 4. Visualize predictions vs actual values
plt.figure(figsize=(14, 6))

# Plot actual test values
plt.plot(actual_test_dates, actual_values, "o-", label="Actual", linewidth=2, markersize=5)

# Plot predictions
plt.plot(actual_test_dates, predictions_series, "s--", label="Seasonal Naive (Weekly)", linewidth=2, markersize=5, alpha=0.7)

plt.xlabel("Date", fontsize=12)
plt.ylabel("Incoming Calls", fontsize=12)
plt.title("Seasonal Naive (Weekly) Baseline Model - Predictions vs Actual", fontsize=14, fontweight="bold")
plt.legend(fontsize=11)
plt.grid(True, alpha=0.3)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

print("\nPrediction accuracy comparison:")
print(f"Min calls: {actual_values.min():.0f}, Max calls: {actual_values.max():.0f}")
print(f"Mean actual: {actual_values.mean():.2f}, Mean predicted: {predictions_series.mean():.2f}")

## Advanced Model: SARIMA (Seasonal AutoRegressive Integrated Moving Average)

In [ ]:
# Install statsmodels if not already installed
import subprocess
import sys

try:
    from statsmodels.tsa.statespace.sarimax import SARIMAX
except ImportError:
    subprocess.check_call([sys.executable, "-m", "pip", "install", "statsmodels"])
    from statsmodels.tsa.statespace.sarimax import SARIMAX

In [ ]:
# Train SARIMA model with simple, proven parameters
# SARIMA(1,1,1)(1,1,1,7) - common good baseline for weekly seasonal data
import warnings

warnings.filterwarnings("ignore")

# Simple SARIMA parameters
order = (1, 1, 1)  # (p, d, q)
seasonal_order = (1, 1, 1, 7)  # (P, D, Q, m) - m=7 for weekly seasonality

print(f"Training SARIMA{order}x{seasonal_order} model...")
print("Parameters: p=1, d=1, q=1 (non-seasonal), P=1, D=1, Q=1, m=7 (seasonal)\n")

sarima_model = SARIMAX(
    train["Incoming Calls"],
    order=order,
    seasonal_order=seasonal_order,
    enforce_stationarity=False,
    enforce_invertibility=False,
)

sarima_results = sarima_model.fit(disp=False)
print("SARIMA Model trained successfully!")

# Make predictions on test set
sarima_predictions = sarima_results.get_forecast(steps=len(test))
sarima_forecast = sarima_predictions.predicted_mean

# Evaluate SARIMA model
sarima_mae = mean_absolute_error(test["Incoming Calls"], sarima_forecast)
sarima_rmse = np.sqrt(mean_squared_error(test["Incoming Calls"], sarima_forecast))
sarima_mape = mean_absolute_percentage_error(test["Incoming Calls"], sarima_forecast)

print("\n" + "=" * 60)
print("SARIMA Model - Evaluation Metrics")
print("=" * 60)
print(f"Mean Absolute Error (MAE):           {sarima_mae:.2f}")
print(f"Root Mean Squared Error (RMSE):      {sarima_rmse:.2f}")
print(f"Mean Absolute Percentage Error (MAPE): {sarima_mape:.4f} ({sarima_mape*100:.2f}%)")
print("=" * 60)

# Visualize predictions vs actual values
plt.figure(figsize=(14, 6))
plt.plot(test.index, test["Incoming Calls"], "o-", label="Actual", linewidth=2, markersize=5)
plt.plot(test.index, sarima_forecast, "s--", label="SARIMA Predictions", linewidth=2, markersize=5, alpha=0.7)
plt.xlabel("Date", fontsize=12)
plt.ylabel("Incoming Calls", fontsize=12)
plt.title("SARIMA Model - Predictions vs Actual", fontsize=14, fontweight="bold")
plt.legend(fontsize=11)
plt.grid(True, alpha=0.3)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

## Advanced Model: Prophet

In [ ]:
# Install prophet if not already installed
import subprocess
import sys

try:
    from prophet import Prophet
except ImportError:
    subprocess.check_call([sys.executable, "-m", "pip", "install", "prophet"])
    from prophet import Prophet

In [ ]:
# Prepare data for Prophet
# Prophet requires columns 'ds' (datestamp) and 'y' (value)
prophet_train = train.reset_index()
prophet_train = prophet_train.rename(columns={"Date": "ds", "Incoming Calls": "y"})

# Initialize and train the model
prophet_model = Prophet(
    seasonality_mode="multiplicative",  # Good for data with trends
    weekly_seasonality=True,
    daily_seasonality=False,
)
prophet_model.fit(prophet_train)

# Create future dataframe for predictions
future = prophet_model.make_future_dataframe(periods=len(test))
forecast = prophet_model.predict(future)

# Extract predictions for the test set period
prophet_forecast = forecast[forecast["ds"].isin(test.index)]

print("Prophet model trained and forecast generated.")
print(f"Forecast length: {len(prophet_forecast)}")

In [ ]:
# Evaluate Prophet model
prophet_mae = mean_absolute_error(test["Incoming Calls"], prophet_forecast["yhat"])
prophet_rmse = np.sqrt(mean_squared_error(test["Incoming Calls"], prophet_forecast["yhat"]))
prophet_mape = mean_absolute_percentage_error(test["Incoming Calls"], prophet_forecast["yhat"])

print("\n" + "=" * 60)
print("Prophet Model - Evaluation Metrics")
print("=" * 60)
print(f"Mean Absolute Error (MAE):           {prophet_mae:.2f}")
print(f"Root Mean Squared Error (RMSE):      {prophet_rmse:.2f}")
print(f"Mean Absolute Percentage Error (MAPE): {prophet_mape:.4f} ({prophet_mape*100:.2f}%)")
print("=" * 60)

In [ ]:
# Visualize Prophet forecast
fig1 = prophet_model.plot(forecast)
plt.title("Prophet Forecast - Full Data View", fontsize=14, fontweight="bold")
plt.xlabel("Date", fontsize=12)
plt.ylabel("Incoming Calls", fontsize=12)
plt.show()

# Plot Prophet components
fig2 = prophet_model.plot_components(forecast)
plt.show()

In [ ]:
# Compare all three models
comparison_df = pd.DataFrame(
    {
        "Seasonal Naive MAE": [mae],
        "Seasonal Naive RMSE": [rmse],
        "Seasonal Naive MAPE": [mape * 100],
        "SARIMA MAE": [sarima_mae],
        "SARIMA RMSE": [sarima_rmse],
        "SARIMA MAPE": [sarima_mape * 100],
        "Prophet MAE": [prophet_mae],
        "Prophet RMSE": [prophet_rmse],
        "Prophet MAPE": [prophet_mape * 100],
    }
)

print("\n" + "=" * 80)
print("MODEL COMPARISON - Baseline vs SARIMA vs Prophet")
print("=" * 80)
print(comparison_df.T)
print("=" * 80)

## Time Series Cross-Validation for SARIMA

In [ ]:
# Time Series Cross-Validation for SARIMA
# This uses a rolling window approach: train on increasing data, test on next period


def time_series_cv_sarima(data, n_splits=5, test_size=7):
    """
    Perform time series cross-validation for SARIMA model

    Args:
        data: Time series data (pandas Series)
        n_splits: Number of cross-validation splits
        test_size: Number of periods to use for testing in each split

    Returns:
        List of MAE scores for each fold
    """

    # Calculate split points
    total_length = len(data)
    min_train_size = total_length - n_splits * test_size

    mae_scores = []
    rmse_scores = []
    mape_scores = []

    print(f"Performing {n_splits}-fold time series cross-validation...")
    print(f"Test size per fold: {test_size} days")
    print(f"Minimum training size: {min_train_size} days\n")

    for i in range(n_splits):
        # Calculate train and test indices
        test_end = total_length - i * test_size
        test_start = test_end - test_size
        train_end = test_start

        # Split data
        cv_train = data.iloc[:train_end]
        cv_test = data.iloc[test_start:test_end]

        print(f"Fold {i+1}/{n_splits}: Train size = {len(cv_train)}, Test size = {len(cv_test)}")

        try:
            # Train SARIMA model
            cv_model = SARIMAX(
                cv_train,
                order=(1, 1, 1),
                seasonal_order=(1, 1, 1, 7),
                enforce_stationarity=False,
                enforce_invertibility=False,
            )
            cv_results = cv_model.fit(disp=False)

            # Make predictions
            cv_forecast = cv_results.get_forecast(steps=len(cv_test))
            cv_pred = cv_forecast.predicted_mean

            # Calculate metrics
            mae = mean_absolute_error(cv_test, cv_pred)
            rmse = np.sqrt(mean_squared_error(cv_test, cv_pred))
            mape = mean_absolute_percentage_error(cv_test, cv_pred)

            mae_scores.append(mae)
            rmse_scores.append(rmse)
            mape_scores.append(mape)

            print(f"  MAE: {mae:.2f}, RMSE: {rmse:.2f}, MAPE: {mape:.4f}")

        except Exception as e:
            print(f"  Error in fold {i+1}: {e}")
            continue

    return mae_scores, rmse_scores, mape_scores


# Perform cross-validation on the training data
cv_mae, cv_rmse, cv_mape = time_series_cv_sarima(train["Incoming Calls"], n_splits=5, test_size=7)

In [ ]:
# Analyze cross-validation results
if cv_mae:  # Check if we have results
    print("\n" + "=" * 60)
    print("TIME SERIES CROSS-VALIDATION RESULTS")
    print("=" * 60)

    print(f"Number of successful folds: {len(cv_mae)}")
    print("\nMAE Scores by Fold:")
    for i, mae in enumerate(cv_mae, 1):
        print(f"  Fold {i}: {mae:.2f}")

    print("\nCross-Validation Summary:")
    print(f"  Mean MAE:  {np.mean(cv_mae):.2f} ± {np.std(cv_mae):.2f}")
    print(f"  Mean RMSE: {np.mean(cv_rmse):.2f} ± {np.std(cv_rmse):.2f}")
    print(f"  Mean MAPE: {np.mean(cv_mape):.4f} ± {np.std(cv_mape):.4f}")

    print("\nComparison with Single Test:")
    print(f"  Single Test MAE:  {sarima_mae:.2f}")
    print(f"  CV Mean MAE:      {np.mean(cv_mae):.2f}")
    print(f"  Difference:       {abs(sarima_mae - np.mean(cv_mae)):.2f}")

    print("=" * 60)
else:
    print("No successful cross-validation folds completed.")